# 1.数据集加载和预处理

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
# 载入Iris数据集
iris = datasets.load_iris()
# print(iris)
X = iris.data # 获取iris数据集data属性的值，二维，行数等于样本数，列数等于特征数
# print(X)
Y = iris.target # 获取iris数据集target属性的值
# print(Y)
# 随机划分训练集与测试集
X_pretrain, X_pretest, y_train, y_test = train_test_split(X, Y, test_size = 0.3)
# 归一化
minMax = MinMaxScaler() # 创建一个minMax对象
X_train = minMax.fit_transform(X_pretrain)
'''
使用minMax对象中的fit_transform方法对训练集做归一化处理
此处使用fit_transform是对训练集的特征的数据做拟合，得到其各个特征的最大值和最小值，然后transform进行归一化处理
'''
X_test = minMax.transform(X_pretest) # 使用minMax对象中的fit_transform方法对测试集做归一化处理
'''
此处使用transform而不是fit_transform的原因：
如果使用fit_transform则是先对测试集的特征的数据做拟合，得到其各个特征的最大值和最小值，然后进行归一处理
这样就会导致得到的X_train和X_test的计算模型不同，所以不使用fit_transform，直接transform
'''

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

'\n此处使用transform而不是fit_transform的原因：\n如果使用fit_transform则是先对测试集的特征的数据做拟合，得到其各个特征的最大值和最小值，然后进行归一处理\n这样就会导致得到的X_train和X_test的计算模型不同，所以不使用fit_transform，直接transform\n'

# 2.导入包与定义结点类

In [31]:
from sklearn.datasets import load_iris
import numpy as np
import math
from collections import Counter

class decisionnode: # 定义结点类
    def __init__(self, d=None, thre=None, results=None, min_sample_number=None, lb=None, rb=None, max_label=None):
        self.d = d # d 表示用于划分的属性索引
        self.thre = thre # thre 表示划分所使用的阈值，每次分裂将样本集分为 2 个子集
        self.results = results # 叶结点所代表的类别
        self.min_sample_number = min_sample_number # 存储分支结点的最小样本量
        self.lb = lb # 左子结点，对应属性值不大于 thre 的那些样本所在的结点
        self.rb = rb # 右子结点，对应属性值大于 thre 的那些样本所在的结点
        self.max_label = max_label # 记录当前结点包含的样本中样本比例最高的类别

min_sample_number = 10 #设置分支结点中包含的最小样本数（即如果一个结点中包含的样本数小于该值则停止分裂）

# 3.计算信息熵
- 首先获取labels列表
- 其次计算信息熵

In [32]:
def entropy(y):
    # 计算信息熵，y为labels，是包含了训练集中每个样本的label值的一个列表
    # 先获取category，即label的去重
    if y.size > 1:  # size()用来获取元素个数。如果y中元素个数大于1则需要使用set()函数先将
        category = list(set(y)) # 获取labels列表：y是训练集中每个样本的label值，set()将用来去重，得到去重后的标签个数，而后将其转成列表
    else: #
        category = [y.item()]  #
        y = [y.item()] # 利用item()对y进行遍历，并将y中的所有元素整合为列表y
    ent = 0
    # 而后在
    for label in category: # 计算信息熵
        p = len([label_ for label_ in y if label_ == label]) / len(y) # p：样本集合中第k类样本所占的比例
        ent += -p * math.log(p, 2) # 计算信息熵，累加
    return ent

# 4.计算基尼值
- 首先获取labels列表
- 其次计算基尼指数

In [33]:
def Gini(y):
    # 计算基尼值，y 为 labels
    category = list(set(y)) # 获取labels列表
    gini = 1
    for label in category:
        p = len([label_ for label_ in y if label_ == label]) / len(y)
        gini += -p * p # 基尼值：1-（两个样本为同一类的概率的累加）
    return gini

# 5.基于最大信息增益计算最优划分阈值
- 利用for循环，基于信息增益，对候选划分点不断迭代，最终得到最大的信息增益和最优阈值

In [34]:
def GainEnt_max(X, y, d):
    # 基于最大信息增益计算最优划分阈值，X 为样本集的属性值，y 为目标值，d 是当前划分使用的属性索引
    ent_X = entropy(y) # 信息熵
    X_attr = X[:, d] # 获取样本集中索引d对应的属性的值
    X_attr = list(set(X_attr)) 
    X_attr = sorted(X_attr) # d是我们使用的分类属性，是连续性数值数据，所以需要对连续型数值进行处理，这里利用sorted()函数对X_attr进行升序排列，以便后续选取中位点对连续值处理
    Gain = 0
    thre = 0
    for i in range(len(X_attr) - 1):
        thre_temp = (X_attr[i] + X_attr[i + 1]) / 2 # 候选划分点
        y_small_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] <= thre_temp] # 获取属性值小于等于候选划分点的索引列表
        y_big_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] > thre_temp] # 获取属性值大于候选划分点的索引列表
        y_small = y[y_small_index] 
        y_big = y[y_big_index] 
        Gain_temp = ent_X - (len(y_small) / len(y)) * entropy(y_small) - (len(y_big) / len(y)) * entropy(y_big) # 计算信息增益
        if Gain < Gain_temp: # 选取信息增益较大的
            Gain = Gain_temp # 信息增益
            thre = thre_temp # 最优阈值
    return Gain, thre

# 6.基于最小基尼指数计算最优划分阈值
- 利用for循环，基于基尼指数，对候选划分点不断迭代，最终得到最小的基尼指数和最优阈值

In [35]:
def Gini_index_min(X, y, d):
    # 基于最小基尼指数计算最优划分阈值，X 为样本集的属性值，y 为目标值，d 是当前划分使用的属性索引
    X_attr = X[:, d] # 获取样本集中索引d对应的属性的值
    X_attr = list(set(X_attr))
    X_attr = sorted(X_attr) # 升序排列
    Gini_index = 1
    thre = 0
    for i in range(len(X_attr) - 1):
        thre_temp = (X_attr[i] + X_attr[i + 1]) / 2 # 候选划分点
        y_small_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] <= thre_temp] # 获取属性值小于等于候选划分点的索引列表
        y_big_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] > thre_temp] # 获取属性值大于候选划分点的索引列表
        y_small = y[y_small_index]
        y_big = y[y_big_index]
        Gini_index_temp = (len(y_small) / len(y)) * Gini(y_small) + (len(y_big) / len(y)) * Gini(y_big) # 计算基尼指数
        if Gini_index > Gini_index_temp: # 选取基尼指数较小的
            Gini_index = Gini_index_temp # 基尼指数
            thre = thre_temp # 最优阈值
    return Gini_index, thre

# 7.基于信息增益选择最优属性

In [36]:
def attribute_based_on_GainEnt(X, y):
    # 基于信息增益选择最优属性，X 为样本集的属性值，y 为目标值
    D = np.arange(len(X[0]))
    Gain_max = 0
    thre_ = 0
    d_ = 0
    for d in D:
        Gain, thre = GainEnt_max(X, y, d) # 计算该属性的最优划分阈值
        if Gain_max < Gain:
            Gain_max = Gain # 选取信息增益较大的
            thre_ = thre # 划分阈值
            d_ = d # 属性索引
    return Gain_max, thre_, d_

# 8.基于基尼指数选择最优属性

In [37]:
def attribute_based_on_Giniindex(X, y):
    # 基于基尼指数选择最优属性，X 为样本集的属性值，y 为目标值
    D = np.arange(len(X[0]))
    Gini_Index_Min = 1
    thre_ = 0
    d_ = 0
    for d in D:
        Gini_index, thre = Gini_index_min(X, y, d) #计算该属性的最优划分阈值
        if Gini_Index_Min > Gini_index:
            Gini_Index_Min = Gini_index # 选取基尼指数较小的
            thre_ = thre # 划分阈值
            d_ = d # 属性索引
    return Gini_Index_Min, thre_, d_

# 9.按照索引为 d 的属性、以 thre 为阈值将数据分为两个子集并返回

In [38]:
def devide_group(X, y, thre, d):
    # 按照索引为 d 的属性、以 thre 为阈值将数据分为两个子集并返回
    X_in_d = X[:, d]
    x_small_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] <= thre] # 获取属性值小于等于候选划分点的索引列表
    x_big_index = [i_arg for i_arg in range(len(X[:, d])) if X[i_arg, d] > thre] # 获取属性值大于候选划分点的索引列表

    X_small = X[x_small_index] 
    y_small = y[x_small_index] 
    X_big = X[x_big_index] 
    y_big = y[x_big_index] 
    return X_small, y_small, X_big, y_big 

# 10.计算样本集中样本占比最多的类别

In [39]:
def maxlabel(y): 
    # 计算样本集中样本占比最多的类别
    label_ = Counter(y).most_common(1) # 利用collections.Counter中的most_common()方法得到样本中出现最多的1个类与次数
    return label_[0][0] # 得到样本中出现次数最多的1个类别

# 11.构建决策树

In [40]:
def buildtree(X, y, method='Gini'):
    # 递归的方式构建决策树
    if y.size > 1:
        if method == 'Gini':
            Gain_max, thre, d = attribute_based_on_Giniindex(X, y) # 上面定义的基于基尼指数选择最优属性的函数
        elif method == 'GainEnt':
            Gain_max, thre, d = attribute_based_on_GainEnt(X, y) # 上面定义的基于信息增益选择最优属性的函数
        if len(list(y)) >= min_sample_number and ((Gain_max > 0 and method == 'GainEnt') or (Gain_max >=0 and method == 'Gini')):
            X_small, y_small, X_big, y_big = devide_group(X, y, thre, d) # 上面定义的划分子集的函数
            left_branch = buildtree(X_small, y_small, method=method) # 左子结点，对应属性值不大于thre的那些样本所在的结点
            right_branch = buildtree(X_big, y_big, method=method) # 右子结点，对应属性值大于thre的那些样本所在的结点
            max_label = maxlabel(y) # 上面定义的计算样本集中样本占比最多的类别的函数，max_label用来记录当前结点包含的样本中样本比例最高的类别
            return decisionnode(d=d, thre=thre, min_sample_number=min_sample_number, lb=left_branch, rb=right_branch, max_label=max_label)
        else:
            max_label = maxlabel(y)
            return decisionnode(results=y[0], min_sample_number=min_sample_number, max_label=max_label)
    else:
        max_label = maxlabel(y)
        return decisionnode(results=y.item(), min_sample_number=min_sample_number, max_label=max_label)

# 12.利用决策树对样本进行分类

In [41]:
def classify(observation, tree): # 利用决策树对样本进行分类
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.d]
        branch = None
        if v > tree.thre: 
            branch = tree.rb # 对应属性值大于thre的那些样本所在的结点
        else:
            branch = tree.lb # 对应属性值小于等于thre的那些样本所在的结点
        return classify(observation, branch)

# 13.调用决策树进行预测

In [42]:
if __name__ == '__main__':
    iris = load_iris()
    X = iris.data # 获取iris数据集data属性的值，二维，行数等于样本数，列数等于特征数
    y = iris.target # 获取iris数据集target属性的值
    np.random.seed(0)
    permutation = np.random.permutation(X.shape[0]) # 使用permutation随机打乱
    shuffled_dataset = X[permutation, :]
    shuffled_labels = y[permutation]

    train_data = shuffled_dataset[:100, :] # 训练数据，所有列的前100行数据
    train_label = shuffled_labels[:100] # 训练目标值，前100行

    test_data = shuffled_dataset[100:150, :] # 测试数据
    test_label = shuffled_labels[100:150] # 测试目标值
 
    tree1 = buildtree(train_data, train_label, method='Gini') # 使用上方定义的buildtree()函数构建决策树1
    tree2 = buildtree(train_data, train_label, method='GainEnt') # 使用上方定义的buildtree()函数构建决策树1

    true_count = 0
    for i in range(len(test_label)):
        predict = classify(test_data[i], tree1) # 利用tree1对test_data进行分类
        if predict == test_label[i]:
            true_count += 1
    acc=true_count/len(test_label) # 计算准确率
    print("CARTTree:{}".format(acc))
    
    true_count = 0
    for i in range(len(test_label)):
        predict = classify(test_data[i], tree2) # 利用tree2对test_data进行分类
        if predict == test_label[i]:
            true_count += 1
    acc=true_count/len(test_label)
    print("C3Tree:{}".format(acc)) # 计算准确率

CARTTree:0.96
C3Tree:0.96


程序运行结束后，可在屏幕上输出 CART 决策树（使用基尼指数）和 C3 决策树（使用信息增益）在测试集上的分类准确率：  
CARTTree:0.96  
C3Tree:0.96